In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
%matplotlib inline


In [ ]:
df_results = pd.read_csv('data/results.csv')
df_results = df_results[['raceId', 'driverId', 'constructorId', 'grid', 'position']]

df_constructors = pd.read_csv('data/constructors.csv')

df_drivers = pd.read_csv('data/drivers.csv')

df_races = pd.read_csv('data/races.csv')




,raceId,driverId,constructorId,grid,position,driverRef,constructorRef,circuitId,year,round
0,18,1,1,1,1,hamilton,mclaren,1,2008,1
1,18,2,2,5,2,heidfeld,bmw_sauber,1,2008,1
2,18,3,3,7,3,rosberg,williams,1,2008,1
3,18,4,4,11,4,alonso,renault,1,2008,1
4,18,5,1,3,5,kovalainen,mclaren,1,2008,1
...,...,...,...,...,...,...,...,...,...,...
26754,1144,825,210,14,16,kevin_magnussen,haas,24,2024,24
26755,1144,859,215,12,17,lawson,rb,24,2024,24
26756,1144,822,15,9,\N,bottas,sauber,24,2024,24
26757,1144,861,3,20,\N,colapinto,williams,24,2024,24


In [68]:
df_merge = df_results.merge(df_drivers[['driverId', 'driverRef']], on='driverId', how='left')
df_merge = df_merge.merge(df_constructors[['constructorId', 'constructorRef']], on='constructorId', how='left')
df_merge = df_merge.merge(df_races[['raceId', 'circuitId', 'year', 'round']], on='raceId', how='left')
df_merge


df_merge['podium'] = df_merge['position'].isin(['1', '2', '3'])
df_merge = df_merge.sort_values(['year', 'round'])

# filter to after 1980
df_merge = df_merge[df_merge['year'] > 1980]

# last year podium rate
def get_last_year_podium_rate(group):
    result = []
    for idx, row in group.iterrows():
        current_year = row['year']
        last_year_data = group[(group['year'] == current_year - 1)]
        if len(last_year_data) > 0:
            rate = last_year_data['podium'].mean()
        else:
            rate = np.nan
        result.append(rate)
    return pd.Series(result, index=group.index)

df_merge['podium_rate_last_year'] = df_merge.groupby('driverId').apply(get_last_year_podium_rate).reset_index(level=0, drop=True)

# current year podium rate up to last race
def get_current_year_podium_rate(group):
    result = []
    for idx, row in group.iterrows():
        current_year = row['year']
        current_round = row['round']
        current_year_data = group[(group['year'] == current_year) & (group['round'] < current_round)]
        if len(current_year_data) > 0:
            rate = current_year_data['podium'].mean()
        else:
            rate = np.nan
        result.append(rate)
    return pd.Series(result, index=group.index)

df_merge['podium_rate_curr_year'] = df_merge.groupby('driverId').apply(get_current_year_podium_rate).reset_index(level=0, drop=True)

# 3. all time podium rate til current race
df_merge['podium_rate_all_time'] = df_merge.groupby('driverId')['podium'].apply(
    lambda x: x.expanding().mean().shift(1)
).reset_index(level=0, drop=True)

# drop unnecessary columns
df_merge = df_merge.drop(columns=['raceId', 'driverId', 'constructorId', 'circuitId', 'position'])
df_merge.reset_index()



C:\Users\danie\AppData\Local\Temp\ipykernel_16632\1429577800.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_merge['podium_rate_last_year'] = df_merge.groupby('driverId').apply(get_last_year_podium_rate).reset_index(level=0, drop=True)
C:\Users\danie\AppData\Local\Temp\ipykernel_16632\1429577800.py:42: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_merge['podium_rate_curr_year'] = df_merge.groupby('driverId

,index,grid,driverRef,constructorRef,year,round,podium,podium_rate_last_year,podium_rate_curr_year,podium_rate_all_time
0,11544,2,jones,williams,1981,1,True,NaN,NaN,NaN
1,11545,3,reutemann,williams,1981,1,True,NaN,NaN,NaN
2,11546,4,piquet,brabham,1981,1,True,NaN,NaN,NaN
3,11547,6,mario_andretti,alfa,1981,1,False,NaN,NaN,NaN
4,11548,8,cheever,tyrrell,1981,1,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
18444,26754,14,kevin_magnussen,haas,2024,24,False,0.000000,0.000000,0.005405
18445,26755,12,lawson,rb,2024,24,False,0.000000,0.000000,0.000000
18446,26756,9,bottas,sauber,2024,24,False,0.000000,0.000000,0.272358
18447,26757,20,colapinto,williams,2024,24,False,NaN,0.000000,0.000000


In [69]:
# split data into train validation and test

from sklearn.model_selection import train_test_split


df_full_train, df_test = train_test_split(df_merge, test_size=0.2)
df_train, df_val = train_test_split(df_full_train, test_size=0.25)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


y_train = (df_train.podium).astype('int').values
y_val = (df_val.podium).astype('int').values
y_test = (df_test.podium).astype('int').values
y_full_train = (df_full_train.podium).astype('int').values




In [70]:
# imports

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
